# Inference and analysis of cell-cell communication using CellAgentChat

This tutorial provides a step-by-step guide on performing inference, analysis, and visualization of a cell-cell communication network for a single dataset using CellAgentChat. We demonstrate the diverse capabilities of CellAgentChat by applying it to scRNA-seq data obtained from cells of human squamous cell carcinoma patients.

CellAgentChat relies on user-provided gene expression data of cells and utilizes prior knowledge of signaling ligand-receptor interactions to model the probability of cell-cell communication. In this tutorial, we also showcase the incorporation of spatial transcriptomics data, which allows the model to consider the spatial proximity between cells when calculating cell-cell communication.

CellAgentchat also provides an animation framework to view the interaction receiving strength of individual cells in real time. Once the intercellular communication network is inferred, CellAgentChat offers additional functionality for exploring, analyzing, and visualizing the data.

For this tutorial we will use the Visium heart dataset provided by SpatialScope and perform spot deconvolution to get true single-cell reoslution. Then we will use CellAgentChat to get CCIs on each cell.

### Load Packages

In [ ]:
from preprocessor import *
from trajectory import *
import model_setup
from model_setup import *
import scanpy as sc
import permutations
from permutations import *
import Communication
from Communication import *
import abm
from abm import *
import bckground_distribution
from bckground_distribution import *

## Part 1: Data input, processing and initialization

### Load Input Files

CellAgentChat requires two inputs: one is the gene expression data of cells, and the other is user assigned cell labels (i.e cell types/clusters). Optionally, the user can supply the spatial coordinates of cells, the pseudotime trajectory of cell (see pseudotime tutorial) and x_umap (optional, used for plotting). 

The required data are stored in the anndata file provided by SpatialScope which can be found [here](https://drive.google.com/drive/folders/1PXv_brtr-tXshBVEd_HSPIagjX9oF7Kg).

In [ ]:
adata = sc.read_h5ad("./demo_data/V1_Human_Heart_spatial.h5ad")

### Preprocess data

Normalized data (e.g., cells and gene filtered, library-size normalization and then log-transformed with a pseudocount of 1) is required as input for CellAgentChat analysis. If user provides count data, we provide an ```expression_processor``` function. In our example, the data is already normalized.

In [ ]:
adata = expression_processor(adata, normalize = True)

## Part 2: Model Setup

### Loading Database

The default database is derived from CellTalkDB which is a manually curated database of literature-supported ligand-receptor interactions in humans and mouse. CellTalkDB contains almost 3400 validated molecular human interactions and over 2000 validated mouse interactions.

Change ```file = 'mouse_lr_pair.tsv'``` for mouse database.

Users can upload their own database file with curated ligand-receptor pairs.

In [ ]:
lig_uni, rec_uni, lr_pairs = load_db(adata, file = 'human_lr_pair.tsv',sep='\t')
#'../Revision/new_ccdb.csv'

CellAgentChat incorporates prior knowledge of TF-receptor and TF-gene ```human``` or ```mouse``` interactions from pre-existing databases to create a partially connected feedforward network.

In [ ]:
tf_uni, rec_tf_uni = load_tf_db("human", adata, rec_uni)

## Part 3: Spot Deconvolution Using SpatialScope 

### Preprocess snRNA-seq Reference Dataset 

The scRNAseq reference should from the exact same tissue that contains all the celltypes you expect to find in ST spots.

Since we are using a human heart dataset, we will use a human heart snRNA-seq atlas as reference dataset, the raw dataset in h5ad format (global_raw.h5ad) is available in [here](https://www.heartcellatlas.org/).

SpatialScope also provided the pre-pocessed snRNA-ref (Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad) as well as other relevent materials involved in the following example in [here](https://drive.google.com/drive/folders/1PXv_brtr-tXshBVEd_HSPIagjX9oF7Kg), so you can skip this part if using this dataset. 

NOTE: Some of these steps are specific to the reference dataset used in the tutorial. 

In [ ]:
ad_sc = sc.read('global_raw.h5ad')

ad_sc = ad_sc[ad_sc.obs['cell_type']!='doublets']   
ad_sc = ad_sc[ad_sc.obs['cell_type']!='NotAssigned']  
ad_sc = ad_sc[ad_sc.obs['cell_type']!='Mesothelial']  
ad_sc = ad_sc[ad_sc.obs['cell_source']=='Sanger-Nuclei']  

cell_type_column = 'cell_type'

sc.pp.filter_cells(ad_sc, min_counts=500)
sc.pp.filter_cells(ad_sc, max_counts=20000)
sc.pp.filter_genes(ad_sc, min_cells=5)
ad_sc = ad_sc[:,~np.array([_.startswith('MT-') for _ in ad_sc.var.index])]
ad_sc = ad_sc[:,~np.array([_.startswith('mt-') for _ in ad_sc.var.index])]

ad_sc = ad_sc[ad_sc.obs['donor']=='D2'].copy() # reduce batch effect among doners
ad_sc = ad_sc[ad_sc.obs.index.isin(ad_sc.obs.groupby('cell_type').apply(
    lambda x: x.sample(frac=3000/x.shape[0],replace=False) if x.shape[0]>3000 else x).reset_index(level=0,drop=True).index)].copy()

SpatialScope recommends using the top 1000 Highly Variable genes as well as the top 50 marker genes for each cell type. We will also include the ligands and receptors since we are focused on CCI. 

In [ ]:
ad_sc.raw = ad_sc.copy()
sc.pp.normalize_total(ad_sc,target_sum=2000)

sc.pp.highly_variable_genes(ad_sc, flavor='seurat_v3',n_top_genes=1000)
sc.tl.rank_genes_groups(ad_sc, groupby=cell_type_column, method='wilcoxon')
markers_df = pd.DataFrame(ad_sc.uns["rank_genes_groups"]["names"]).iloc[0:100, :]
markers = list(np.unique(markers_df.melt().value.values))
markers = list(set(ad_sc.var.loc[ad_sc.var['highly_variable']==1].index)|set(markers)) # highly variable genes + cell type marker genes

lr_genes = list(lig_uni.keys()) + list(rec_uni.keys())

markers = markers+add_genes+ligand_recept

In [ ]:
ad_sc.var.loc[ad_sc.var.index.isin(markers),'Marker'] = True
ad_sc.var['Marker'] = ad_sc.var['Marker'].fillna(False)
ad_sc.var['highly_variable'] = ad_sc.var['Marker']

sc.pp.log1p(ad_sc)

In [ ]:
ad_sc.write("Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad")

### Train Model 

NOTE: make sure to update paths

Four GPUs used to train scRNA-seq reference in parallel.

    python ./src/Train_scRef.py 
        --ckpt_path ./Ckpts_scRefs/Heart_D2
        --scRef ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type 
        --gpus 0,1,2,3 
        --sigma_begin 50 
        --sigma_end 0.002 
        --step_lr 3e-7 


The checkpoints and sampled psuedo-cells will be saved in ./Ckpts_scRefs, e.g, model_5000.pt, model_5000.h5ad


SpatialScope has provided the pre-trained checkpoint (Ckpts_scRefs/model_5000.pt) for this dataset in [here](https://drive.google.com/drive/folders/1PXv_brtr-tXshBVEd_HSPIagjX9oF7Kg), so you can skip this part if using this dataset

### Run SpatialScope

NOTE: Make sure to update paths

#### Step1: Nuclei segmentation

    python ./src/Nuclei_Segmentation.py 
        --tissue heart 
        --out_dir  ./output  
        --ST_Data ./demo_data/V1_Human_Heart_spatial.h5ad 
        --Img_Data  ./demo_data/V1_Human_Heart_image.tif

#### Step2: Cell Type Identification

    python ./src/Cell_Type_Identification.py 
        --tissue heart 
        --out_dir  ./output  
        --ST_Data ./output/heart/sp_adata_ns.h5ad 
        --SC_Data ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type

#### Step3: Gene Expression Decomposition

SpatialScope is limited by GPU memory, so it can only handle at most about 1000 spots in 4 GPUs at a time

    python ./src/Decomposition.py 
        --tissue heart 
        --out_dir  ./output 
        --SC_Data ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type  
        --ckpt_path ./Ckpts_scRefs/Heart_D2/model_5000.pt 
        --spot_range 0,1000 --gpu 0,1,2,3

    python ./src/Decomposition.py 
        --tissue heart 
        --out_dir  ./output 
        --SC_Data ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type  
        --ckpt_path ./Ckpts_scRefs/Heart_D2/model_5000.pt 
        --spot_range 1000,2000 
        --gpu 0,1,2,3
        
    python ./src/Decomposition.py 
        --tissue heart 
        --out_dir  ./output 
        --SC_Data ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type  
        --ckpt_path ./Ckpts_scRefs/Heart_D2/model_5000.pt 
        --spot_range 2000,3000 --gpu 0,1,2,3

    python ./src/Decomposition.py 
        --tissue heart 
        --out_dir  ./output 
        --SC_Data ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type  
        --ckpt_path ./Ckpts_scRefs/Heart_D2/model_5000.pt 
        --spot_range 3000,4000 
        --gpu 0,1,2,3

    python ./src/Decomposition.py 
        --tissue heart 
        --out_dir  ./output 
        --SC_Data ./Ckpts_scRefs/Heart_D2/Ref_Heart_sanger_D2.h5ad 
        --cell_class_column cell_type  
        --ckpt_path ./Ckpts_scRefs/Heart_D2/model_5000.pt 
        --spot_range 4000,4220 
        --gpu 0,1,2,3

### Arrange Deconvoluted Spots into One Anndata File

In [ ]:
ad_scsts_list = []
ad_scsts_list.append(sc.read('output/heart/generated_cells_spot0_1000.h5ad'))
ad_scsts_list.append(sc.read('output/heart/generated_cells_spot1000_2000.h5ad'))
ad_scsts_list.append(sc.read('output/heart/generated_cells_spot2000_3000.h5ad'))
ad_scsts_list.append(sc.read('output/heart/generated_cells_spot3000_4000.h5ad'))
ad_scsts_list.append(sc.read('output/heart/generated_cells_spot4000_4220.h5ad'))

adata_decon = ad_scsts_list[0].concatenate(
        ad_scsts_list[1:],
        batch_key="_",
        uns_merge="unique",
        index_unique=None
        )

Now we have our final anndata file with deconvolved spots to get true single-cellr esolution. We will now use this anndata for the rest of the tutorial to calculate CCIs using CellAgentChat

For more info on using SpatialScope for spot dconvolution, check their tutorial [here](https://github.com/YangLabHKUST/SpatialScope/blob/master/demos/Human-Heart.ipynb).

## Part 4: Continue Model Setup 

### Deep Learning Model to estimate downstream expression

Setup and train neural network. The neural network takes as input a vector vr<sub>i</sub> representing expression of all receptors in cell i. This receptor expression vector is then fed into the neural network (F) to predict the expression x<sub>i</sub>  of all genes in cell i (regression task). The neural network structure was composed the following components: one input layer, one hidden layer, and output layer. The input layer is of size equivalent to the number of receptors. The output layer is the same size as the number of all genes. The hidden layer is of size one thousand representing transcription factors involved in downstream signaling pathways.

The neural network takes as input two vectors, x<sub>j</sub><sup>R</sup>, representing expression of all receptors in cell j and $\frac{1}{N}$ $\sum_{i=1}^N$ x<sub>i</sub><sup>L</sup>, representing the average expression of all ligands across the set of all N cells. These two vectors are then fed into the neural network (f) to predict the expression $\hat{x}_j^R$ of all genes in cell j (regression task). The neural network structure comprises the following components: two input layers, two hidden layers, and one output layer. The size of the two input layers corresponds to the number of ligands and receptors, respectively. The first hidden layer is dedicated to representing specific LR pairs. Each node in this layer receives two connections, one corresponding to a ligand and the other to a receptor, thus encapsulating the unique LR pair interactions. The second hidden layer represents transcription factors involved in downstream signaling pathways. We leverage scSeqComm, a database that incorporates insights from KEGG and REACTOME pathways to delineate the association between a receptor and its downstream transcription factors (TFs), to inform the connections between this layer and the previous one. The output layer is responsible for outputting downstream gene expression predictions. To establish connections between known TFs and their downstream gene targets, we utilize prior knowledge from databases such as TRRUST v2, HTRIdb, and RegNetwork. If specific information regarding the target genes of certain TFs is unavailable, we utilize dense connections.

The model is saved to the path provided in the ```path``` argument.

In [ ]:
mat, C = train(adata_decon, lig_uni, rec_uni, tf_uni, rec_tf_uni, lr_pairs, path="model.pt")

Load deep learning model

In [ ]:
model_dl = load_model("model.pt")

### Caculate conversion rates for each receptor

With the trained neural network regressor, we then perform sensitivity analysis to score each input receptor based on how much impact it imposes on the downstream gene expression. This is known as the conversion rate for a receptor. 

In [ ]:
conversion_rates = feature_selection(model = model_dl, mat=mat, C=C, rec_uni=rec_uni)

Save conversion rates

In [ ]:
save_conversion_rate(conversion_rates, file='conversion_rates.txt')

Load conversion rates

In [ ]:
conversion_rates = load_conversion_rate(file='conversion_rates.txt')
rates = add_rates(conversion_rates, rec_uni)

### Background Distribution

We employed a random permutation test to calculate the statistical significance p-value associated with each ligand-receptor interaction score. The significant interactions between two cell groups are identified using a permutation test by randomly permuting the group labels of cells and then recalculating the interaction score for each ligand receptor. After each permutation the ligand-receptor score is stored regardless of what cell type pair it came from. 

The ```threshold``` argument sets the minimum number of score values needed on average for each ligand-receptor pair. We recommend atleast a ```threshold=10000``` or greater.

```dist=True```  specifies that we want to consider the spatial coordinates of cells when infering cellular interaction. This leads to the calculation of the average distance between cells, which will be used in the subsequent steps. When spatial information is not provided ```dist=False``` should be set. 

```N``` specifies the number of cells to include in the calculation. By defualt all the cells are included, but the user can specify to use less. 

In [ ]:
test_scores, model2, distance = permutation_test(threshold = 10000, N=len(adata_decon.obs), adata=adata_decon, 
                                                 lig_uni=lig_uni, rec_uni=rec_uni, rates=rates, dist=True)

The average distance between all cells

In [ ]:
distance

For each ligand-receptor pair, we fit the random score distribution with a gamma distribution. By fitting the observed interaction scores (for a specific Ligand-Receptor) pair to a Gamma distribution, we get the estimated parameters (alpha and scale) of the gamma distribution to represent the random interaction score distribution of the Ligand-Receptor pair. 

Due to the computational expense of calculating the distances between all cells in the dataset, the permutation test can be quite costly, especially when considering spatial information. To address this, we have devised a method to match the background gamma distribution of non-spatial calculations with that of spatial ones by scaling the parameters. If the parameters must be scaled the ```dist``` parameter must be given the average distance between the  cells and the ```scale``` parameter must be set to ```True```. 

In [ ]:
params = get_distribution(test_scores, dist=distance, scaled=True)

Save the background distribution for each ligand to the given path

In [ ]:
save_distribution(params, path='new_distribution.csv')

Load the background distribution for each ligand to the given path

In [ ]:
params = load_distribution(file='new_distribution.csv')

### Get choices

When inferring cell communication, the following three lists are used to track specific receptors, ligand-receptor pairs, or cell types:

```proteins``` contains all the receptors in database. 

```pairs``` contains all the ligand-receptor pairs in the database. 

```clusters``` contains all the cell types/clusters present in the dataset. 

In [ ]:
proteins = get_protein_choices(rec_uni)
pairs = get_lr_choices(rec_uni, lig_uni)
clusters = get_cluster_choices(adata_decon)

## Part 5: Inference of cell-cell communication network

We provide a function ```CCI``` calculate cell-cell interactions. 

Optional Parameters:

```delta```: Influences the degree of cell-to-cell distance. For long-range mode, a delta value less than 1 is used. This promotes interactions over long distance ranges. While for short-range mode, a delta value greater than 1 is employed, promoting interactions over closer distance ranges [1].

```max_steps```: Number of iterations to be performed in the simulation [1]. 

```tau```: The degree of the freedom for the distance [2].

```rec_block```: Receptor to be obstructed by in-silico receptor blocking. Interactions involving the chosen receptor will not occur [False].

```plot_every_step```: Whether to plot results after every step [True].

```path```: output path/directory [/out].

```interaction_matrix```: Name of the interaction_matrix results file [interaction_matrix.csv].

```sig_lr_pair```: Name of the file consisting of all the inferred cell-cell communications at the level of ligands/receptors [sig_lr_pair.csv].

```pvalues_name```: Name of the file consisting of the pvalues for the interactions that correspond to the ```sig_lr_pair``` file [pvalues.csv]. 

```pvalues2_names```: Name of the file consisting of the pvalue group (0.05 < p < 0.025, 0.025< p < 0.01, p < 0.01) for the interactions that correspond to the ```sig_lr_pair``` file [pvalues2.csv]. 

```cluster_name```: Name of the file that stores the list of cell types(for plotting) [cluster_names.csv].

```threshold```: The pvalue threshold for significant interactions [0.05]

```net```: Neural network model path

In [ ]:
abm_model = CCI(N=len(adata_decon.obs), adata=adata_decon, lig_uni=lig_uni, rec_uni=rec_uni, max_steps=1,
    rates=rates, distribution=params, clusters=clusters, dist=True, threshold=0.05, net="model.pt")

The CCI function calculates all significant ligand-receptor interactions and plots a heatmap and dotplot of the results. 

Optionally if all the results are obtained, plotting the results can be done separately.

In [ ]:
plotting()

## Part 6: Animation platform

Model parameters described in part 5 and in our animation tutorial video.

In [ ]:
model_params = abm_copy.parameters(adata_decon, lig_uni, rec_uni, rates, clusters, pairs, proteins)

Call the animation platform. This will produce a popup to the animation server. If not considering spatial information ```dist_on``` must be set to ```False```. Otherwise ```dist_on=True```. 

Note: The animation platform needs to be re opened with a different ```port``` number at every use (as long as the same jupyter notebook file is open).  

In [ ]:
abm.visualization(adata_decon, model_params, dist_on=True, port=8521)